In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
import math
from scipy.spatial.distance import euclidean

In [4]:
def EuclideanCal(x):
    lenth = len(x)
    euc = []
    for i in range(lenth):
        if i == 0:
            ed = euclidean(x[i], x[i + 1])
            euc.append(ed)
            continue
        if i == lenth - 1:
            ed = euclidean(x[i], x[i - 1])
            euc.append(ed)
            continue
        ed1 = euclidean(x[i], x[i + 1])
        ed2 = euclidean(x[i], x[i - 1])
        ed = 0.5 * (ed1 + ed2)
        euc.append(ed)

    return np.array(euc)

In [5]:
x1 = np.load("../attention_GRU/dataset/trainset/data/Bearing1_1.npy")
x2 = np.load("../attention_GRU/dataset/trainset/data/Bearing1_2.npy")

pca = PCA(n_components=2)
x1_pca = pca.fit_transform(x1)
x2_pca = pca.fit_transform(x2)
x1_hi= EuclideanCal(x1_pca)
x2_hi = EuclideanCal(x2_pca)
sc = MinMaxScaler(feature_range=(0,1))
x1_hi = sc.fit_transform(x1_hi.reshape((-1,1)))
x2_hi = sc.fit_transform(x2_hi.reshape((-1,1)))

x_hi = np.vstack((x1_hi,x2_hi))

x_train = []
y_train = []

for i in range(20,len(x_hi)):
    x_train.append(x_hi[i-20:i])
    y_train.append(x_hi[i])
    
x_train , y_train = np.array(x_train) , np.array(y_train)
print(x_train.shape , y_train.shape)

np.random.seed(7)
np.random.shuffle(x_train)
np.random.seed(7)
np.random.shuffle(y_train)
tf.random.set_seed(7)

(3654, 20, 1) (3654, 1)


In [6]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dropout, Dense, GRU,LSTM,Conv1D,AveragePooling1D
from sklearn.metrics import mean_squared_error, mean_absolute_error


def scaled_dot_product_attention(q , k , v ,mask):
    """

    :param q: 请求的形状 == {... , seq_len ,depth}
    :param k: 请求的形状 == {... , seq_len ,depth}
    :param v: 数值的形状 == (..., seq_len_v, depth_v)
    :param mask: Float 张量，其形状能转换成
          (..., seq_len_q, seq_len_k)。默认为None。
    :return: 输出，注意力权重
    """
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # 缩放 matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # 将 mask 加入到缩放的张量上。
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

        # softmax 在最后一个轴（seq_len_k）上归一化，因此分数
    # 相加等于1。
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

def print_out(q, k, v):
    temp_out, temp_attn = scaled_dot_product_attention(
        q, k, v, None)
    print ('Attention weights are:')
    print (temp_attn)
    print ('Output is:')
    print (temp_out)


class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """分拆最后一个维度到 (num_heads, depth).
        转置结果使得形状为 (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, q ,k ,v , mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


class attention_model(Model):
    def __init__(self):
        super(attention_model, self).__init__()
        self.layer1 = tf.keras.Sequential([
            Conv1D(filters=80, activation='relu', kernel_size=4, padding='same', strides=1),
            AveragePooling1D(pool_size=1, strides=None, padding='same', data_format='channels_last'),
            Conv1D(filters=80, kernel_size=4, activation='relu', padding='same', strides=1),
            AveragePooling1D(pool_size=1, strides=None, padding='valid', data_format='channels_last'),
            Dense(80),
            Dropout(0.5)
        ])

        self.mha = MultiHeadAttention(d_model=80, num_heads=8)

        self.layer2 = tf.keras.Sequential([
            Dense(80, activation='relu'),
            Dense(80, activation='relu'),
            Dropout(0.5)
        ])

        self.layer3 = tf.keras.Sequential([
            GRU(80),
            Dropout(0.2),
            Dense(1)
        ])

    def call(self, x):
        x = self.layer1(x)
        x, _ = self.mha(x, k=x, v=x, mask=None)
        x = self.layer2(x)
        x = self.layer3(x)

        return x

In [7]:
model = attention_model()
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(0.001))

In [8]:
checkpoint_save_path = "./checkpoint/SACGNet01.ckpt"

if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor='loss')

history = model.fit(x_train, y_train, batch_size=128, epochs=100,callbacks=[cp_callback])

Epoch 1/100
29/29 [==============================] - 1s 33ms/step - loss: 0.0014
Epoch 2/100
29/29 [==============================] - 1s 33ms/step - loss: 0.0013
Epoch 3/100
29/29 [==============================] - 1s 31ms/step - loss: 0.0013
Epoch 4/100
29/29 [==============================] - 1s 34ms/step - loss: 0.0013
Epoch 5/100
29/29 [==============================] - 1s 31ms/step - loss: 0.0013
Epoch 6/100
29/29 [==============================] - 1s 33ms/step - loss: 0.0012
Epoch 7/100
29/29 [==============================] - 1s 33ms/step - loss: 0.0011
Epoch 8/100
29/29 [==============================] - 1s 31ms/step - loss: 0.0014
Epoch 9/100
29/29 [==============================] - 1s 32ms/step - loss: 0.0013
Epoch 10/100
29/29 [==============================] - 1s 32ms/step - loss: 0.0014
Epoch 11/100
29/29 [==============================] - 1s 31ms/step - loss: 0.0014
Epoch 12/100
29/29 [==============================] - 1s 31ms/step - loss: 0.0014
Epoch 13/100
29/29 [=====

In [9]:
x_complete = np.load("..//attention_GRU//dataset//testset//data//Bearing1_3Fulltestset.npy")
x_complete = pca.fit_transform(x_complete)
x_complete = EuclideanCal(x_complete)
x_complete = sc.fit_transform(x_complete.reshape((-1,1)))
complete_len = len(x_complete)  #完整的数据的长度
print("complete_len:",complete_len)

complete_len: 2375


In [11]:
x_test = np.load("..//attention_GRU//dataset//testset//data//Bearing1_3Fulltestset.npy")
x_test = x_test[1782:1802]
x_test = pca.fit_transform(x_test)
x_test = EuclideanCal(x_test)
x_test = sc.fit_transform(x_test.reshape((-1,1)))
print("x_test:",x_test.shape)
test_set = np.reshape(x_test,(1,len(x_test),1))
test_set = np.array(test_set)

x_test: (20, 1)


In [12]:
model = attention_model()
checkpoint_save_path = "./checkpoint/SACGNet01.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

-------------load the model-----------------


In [14]:
def call(inputs, model, times):
    predictions = np.reshape(inputs,(20,1)).tolist()

    predictions.append(model.predict(inputs))

    for n in range(times):
        test = np.reshape( np.array(predictions[-20:],dtype=np.float32), (1,20,1))
        pre = model.predict(test)
        predictions.append(pre)

    return np.array(predictions,dtype=np.float32)

def mape_loss_func(preds,labels):
    mask=labels!=0
    return np.fabs((labels[mask]-preds[mask])/labels[mask]).mean()

In [15]:
# 调用call函数，自回归预测，函数返回一个预测序列给pre，归一化
pre = call(inputs=test_set,model=model,times=600)
#print("pre返回:",pre.shape)
pre = sc.fit_transform(pre)
# pre_data 在不同的轴承需要修改，给的部分轴承序列长度-20
pre_data = x_complete[0:1782]
print("pre_data:",pre_data.shape)
print("pre:",pre.shape)


# 画图
# x1 = np.linspace( 1, len(pre_data), len(pre_data) )
# x2 = np.linspace( len(pre_data)+1, len(pre)+len(pre_data), len(pre) )
# x3 = np.linspace( 1, len(x_complete), len(x_complete) )
#
# plt.figure(figsize=(20,10))
# plt.plot(x1, pre_data, color="red")
# plt.plot(x2, pre, label='pre',color='black')
# plt.plot(x3, x_complete, label='true',color='red',linewidth=3.0)
# plt.show()

# 计算loss
lenth = complete_len-len(pre_data)
mse = mean_squared_error( x_complete[-lenth:] ,pre[0:lenth] )
rmse = math.sqrt(mse)
mae = mean_absolute_error(x_complete[-lenth:] ,pre[0:lenth]  )
mape = mape_loss_func( pre[0:lenth], x_complete[-lenth:] )
print('mse: %.6f' % mse)
print('rmse: %.6f' % rmse)
print('mae: %.6f' % mae)
print('mape: %.6f' % mape)

pre_data: (1782, 1)
pre: (621, 1)
mse: 0.010169
rmse: 0.100839
mae: 0.040551
mape: 1.300305


In [17]:
x_complete = np.load("..//attention_GRU//dataset//testset//data//Bearing1_4Fulltestset.npy")
x_complete = pca.fit_transform(x_complete)
x_complete = EuclideanCal(x_complete)
x_complete = sc.fit_transform(x_complete.reshape((-1,1)))
complete_len = len(x_complete)  #完整的数据的长度
print("complete_len:",complete_len)

x_test = np.load("..//attention_GRU//dataset//testset//data//Bearing1_4Fulltestset.npy")
x_test = x_test[1119:1139]
x_test = pca.fit_transform(x_test)
x_test = EuclideanCal(x_test)
x_test = sc.fit_transform(x_test.reshape((-1,1)))
print("x_test:",x_test.shape)
test_set = np.reshape(x_test,(1,len(x_test),1))
test_set = np.array(test_set)

# 调用call函数，自回归预测，函数返回一个预测序列给pre，归一化
pre = call(inputs=test_set,model=model,times=400)
#print("pre返回:",pre.shape)
pre = sc.fit_transform(pre)
# pre_data 在不同的轴承需要修改，给的部分轴承序列长度-20
pre_data = x_complete[0:1139]
print("pre_data:",pre_data.shape)
print("pre:",pre.shape)


# 画图
# x1 = np.linspace( 1, len(pre_data), len(pre_data) )
# x2 = np.linspace( len(pre_data)+1, len(pre)+len(pre_data), len(pre) )
# x3 = np.linspace( 1, len(x_complete), len(x_complete) )
#
# plt.figure(figsize=(20,10))
# plt.plot(x1, pre_data, color="red")
# plt.plot(x2, pre, label='pre',color='black')
# plt.plot(x3, x_complete, label='true',color='red',linewidth=3.0)
# plt.show()

# 计算loss
lenth = complete_len-len(pre_data)
mse = mean_squared_error( x_complete[-lenth:] ,pre[0:lenth] )
rmse = math.sqrt(mse)
mae = mean_absolute_error(x_complete[-lenth:] ,pre[0:lenth]  )
mape = mape_loss_func( pre[0:lenth], x_complete[-lenth:] )
print('mse: %.6f' % mse)
print('rmse: %.6f' % rmse)
print('mae: %.6f' % mae)
print('mape: %.6f' % mape)

complete_len: 1428
x_test: (20, 1)
pre_data: (1139, 1)
pre: (421, 1)
mse: 0.052913
rmse: 0.230028
mae: 0.156616
mape: 1.460895


In [18]:
x_complete = np.load("..//attention_GRU//dataset//testset//data//Bearing1_5Fulltestset.npy")
x_complete = pca.fit_transform(x_complete)
x_complete = EuclideanCal(x_complete)
x_complete = sc.fit_transform(x_complete.reshape((-1,1)))
complete_len = len(x_complete)  #完整的数据的长度
print("complete_len:",complete_len)

x_test = np.load("..//attention_GRU//dataset//testset//data//Bearing1_5Fulltestset.npy")
x_test = x_test[2282:2302]
x_test = pca.fit_transform(x_test)
x_test = EuclideanCal(x_test)
x_test = sc.fit_transform(x_test.reshape((-1,1)))
print("x_test:",x_test.shape)
test_set = np.reshape(x_test,(1,len(x_test),1))
test_set = np.array(test_set)

# 调用call函数，自回归预测，函数返回一个预测序列给pre，归一化
pre = call(inputs=test_set,model=model,times=200)
#print("pre返回:",pre.shape)
pre = sc.fit_transform(pre)
# pre_data 在不同的轴承需要修改，给的部分轴承序列长度-20
pre_data = x_complete[0:2282]
print("pre_data:",pre_data.shape)
print("pre:",pre.shape)


# 画图
# x1 = np.linspace( 1, len(pre_data), len(pre_data) )
# x2 = np.linspace( len(pre_data)+1, len(pre)+len(pre_data), len(pre) )
# x3 = np.linspace( 1, len(x_complete), len(x_complete) )
#
# plt.figure(figsize=(20,10))
# plt.plot(x1, pre_data, color="red")
# plt.plot(x2, pre, label='pre',color='black')
# plt.plot(x3, x_complete, label='true',color='red',linewidth=3.0)
# plt.show()

# 计算loss
lenth = complete_len-len(pre_data)
mse = mean_squared_error( x_complete[-lenth:] ,pre[0:lenth] )
rmse = math.sqrt(mse)
mae = mean_absolute_error(x_complete[-lenth:] ,pre[0:lenth]  )
mape = mape_loss_func( pre[0:lenth], x_complete[-lenth:] )
print('mse: %.6f' % mse)
print('rmse: %.6f' % rmse)
print('mae: %.6f' % mae)
print('mape: %.6f' % mape)

complete_len: 2463
x_test: (20, 1)
pre_data: (2282, 1)
pre: (221, 1)
mse: 0.038760
rmse: 0.196877
mae: 0.077266
mape: 5.800110


In [19]:
x_complete = np.load("..//attention_GRU//dataset//testset//data//Bearing1_6Fulltestset.npy")
x_complete = pca.fit_transform(x_complete)
x_complete = EuclideanCal(x_complete)
x_complete = sc.fit_transform(x_complete.reshape((-1,1)))
complete_len = len(x_complete)  #完整的数据的长度
print("complete_len:",complete_len)

x_test = np.load("..//attention_GRU//dataset//testset//data//Bearing1_6Fulltestset.npy")
x_test = x_test[2282:2302]
x_test = pca.fit_transform(x_test)
x_test = EuclideanCal(x_test)
x_test = sc.fit_transform(x_test.reshape((-1,1)))
print("x_test:",x_test.shape)
test_set = np.reshape(x_test,(1,len(x_test),1))
test_set = np.array(test_set)

# 调用call函数，自回归预测，函数返回一个预测序列给pre，归一化
pre = call(inputs=test_set,model=model,times=200)
#print("pre返回:",pre.shape)
pre = sc.fit_transform(pre)
# pre_data 在不同的轴承需要修改，给的部分轴承序列长度-20
pre_data = x_complete[0:2282]
print("pre_data:",pre_data.shape)
print("pre:",pre.shape)


# 画图
# x1 = np.linspace( 1, len(pre_data), len(pre_data) )
# x2 = np.linspace( len(pre_data)+1, len(pre)+len(pre_data), len(pre) )
# x3 = np.linspace( 1, len(x_complete), len(x_complete) )
#
# plt.figure(figsize=(20,10))
# plt.plot(x1, pre_data, color="red")
# plt.plot(x2, pre, label='pre',color='black')
# plt.plot(x3, x_complete, label='true',color='red',linewidth=3.0)
# plt.show()

# 计算loss
lenth = complete_len-len(pre_data)
mse = mean_squared_error( x_complete[-lenth:] ,pre[0:lenth] )
rmse = math.sqrt(mse)
mae = mean_absolute_error(x_complete[-lenth:] ,pre[0:lenth]  )
mape = mape_loss_func( pre[0:lenth], x_complete[-lenth:] )
print('mse: %.6f' % mse)
print('rmse: %.6f' % rmse)
print('mae: %.6f' % mae)
print('mape: %.6f' % mape)

complete_len: 2448
x_test: (20, 1)
pre_data: (2282, 1)
pre: (221, 1)
mse: 0.041904
rmse: 0.204705
mae: 0.078639
mape: 2.706986


In [20]:
x_complete = np.load("..//attention_GRU//dataset//testset//data//Bearing1_7Fulltestset.npy")
x_complete = pca.fit_transform(x_complete)
x_complete = EuclideanCal(x_complete)
x_complete = sc.fit_transform(x_complete.reshape((-1,1)))
complete_len = len(x_complete)  #完整的数据的长度
print("complete_len:",complete_len)

x_test = np.load("..//attention_GRU//dataset//testset//data//Bearing1_7Fulltestset.npy")
x_test = x_test[1482:1502]
x_test = pca.fit_transform(x_test)
x_test = EuclideanCal(x_test)
x_test = sc.fit_transform(x_test.reshape((-1,1)))
print("x_test:",x_test.shape)
test_set = np.reshape(x_test,(1,len(x_test),1))
test_set = np.array(test_set)

# 调用call函数，自回归预测，函数返回一个预测序列给pre，归一化
pre = call(inputs=test_set,model=model,times=800)
#print("pre返回:",pre.shape)
pre = sc.fit_transform(pre)
# pre_data 在不同的轴承需要修改，给的部分轴承序列长度-20
pre_data = x_complete[0:1482]
print("pre_data:",pre_data.shape)
print("pre:",pre.shape)


# 画图
# x1 = np.linspace( 1, len(pre_data), len(pre_data) )
# x2 = np.linspace( len(pre_data)+1, len(pre)+len(pre_data), len(pre) )
# x3 = np.linspace( 1, len(x_complete), len(x_complete) )
#
# plt.figure(figsize=(20,10))
# plt.plot(x1, pre_data, color="red")
# plt.plot(x2, pre, label='pre',color='black')
# plt.plot(x3, x_complete, label='true',color='red',linewidth=3.0)
# plt.show()

# 计算loss
lenth = complete_len-len(pre_data)
mse = mean_squared_error( x_complete[-lenth:] ,pre[0:lenth] )
rmse = math.sqrt(mse)
mae = mean_absolute_error(x_complete[-lenth:] ,pre[0:lenth]  )
mape = mape_loss_func( pre[0:lenth], x_complete[-lenth:] )
print('mse: %.6f' % mse)
print('rmse: %.6f' % rmse)
print('mae: %.6f' % mae)
print('mape: %.6f' % mape)

complete_len: 2259
x_test: (20, 1)
pre_data: (1482, 1)
pre: (821, 1)
mse: 0.011662
rmse: 0.107989
mae: 0.022091
mape: 2.525665
